#### Read csv

In [101]:
import pandas as pd
import json
import re

In [3]:
df = pd.read_csv('data/test.csv')

In [4]:
df.columns

Index(['Authors', 'Author full names', 'Author(s) ID', 'Title', 'Year',
       'Source title', 'Cited by', 'DOI', 'Link', 'Affiliations',
       'Authors with affiliations', 'Author Keywords', 'Index Keywords',
       'Editors', 'Publisher', 'ISSN', 'ISBN', 'CODEN', 'PubMed ID',
       'Language of Original Document', 'Document Type', 'Source', 'EID'],
      dtype='object')

In [126]:
# Function to extract IDs between parentheses
def extract_ids(text):
    return re.findall(r'\((\d+)\)', text)

In [133]:
# Load the cities dictionary from the JSON file
with open('cities_mapping.json', 'r', encoding='utf-8') as json_file:
    cities_mapping = json.load(json_file)
# cities_mapping

In [134]:
def extract_city_name(affiliation_full_name):
    
    # for each key in the dict, if the key exists return the value
    for key, value in cities_mapping.items():
        if key in affiliation_full_name.lower():
            return value
    
    return None

In [135]:
def extract_country_name(affiliation_full_name):
    try:
        parts = affiliation_full_name.split(',')
        country = parts[-1].strip().lower()
        return country
    except:
        return None

In [136]:
def duplicate_rows_by_author(df):
    new_df = pd.DataFrame(columns=["author_id", "author_name", "affiliation_full_name", "city", "country"])

    for _, row in df.iterrows():
        authors_with_ids = row["Author full names"].split(';')
        authors_with_affiliations = row["Affiliations"].split(';')
        ids, author_names, affiliations, cities, countries = [], [], [], [], []
        
        for auth_id_name, aff in zip(authors_with_ids, authors_with_affiliations):
            # extract author name
            name_pattern = r"[A-Za-záéíóúÁÉÍÓÚñÑ]+,\s[A-Za-záéíóúÁÉÍÓÚñÑ]+"
            name_match = re.search(name_pattern, auth_id_name)
            # author_name = auth_id.split('.')[0].strip()

            # extract id
            id_match = re.search(r'\((\d+)\)', auth_id_name)
            if id_match and name_match:
                author_id = id_match.group(1)
                author_name = name_match.group()
            else:
                continue

            # extract city and country
            city = extract_city_name(aff)
            country = extract_country_name(aff)
            if country not in ["morocco", "maroc"]:
                continue

            ids.append(author_id)
            author_names.append(author_name)
            affiliations.append(aff)
            cities.append(city)
            # cities1.append(city1)
            countries.append(country)

        data = pd.DataFrame({
            "author_id": ids,
            "author_name": author_names,
            "affiliation_full_name": affiliations,
            "city": cities,
            "country": countries
        })

        new_df = pd.concat([new_df, data], ignore_index=True)
        
    return new_df

In [137]:
trans_df = duplicate_rows_by_author(df)
trans_df.to_csv("metadata.csv", index=False)
trans_df

,author_id,author_name,affiliation_full_name,city,country
0,23004567900,"Khachane, M","Laboratoire de Chimie du Solide Minéral, dépar...",Marrakech,morocco
1,6701835701,"Villain, S",Laboratoire de physique du Solide et couches ...,Marrakech,morocco
2,12782205700,"Mjahed, Mostafa","Ecole Royale de l'Air, Maths and Systems Dept....",Marrakech,morocco
3,56698346700,"Sáenz, L","Departement de Biologie, Université Ibn Tofaï...",Kénitra,morocco
4,56615820600,"Bahi, S","Mohammed v University, Faculty of Sciences, Ra...",Rabat,morocco
...,...,...,...,...,...
150,23978044400,"Elghazi, Haddou","Solid State Physics Laboratory, Faculty of Sci...",Fès,morocco
151,7006857659,"Jorio, Anouar","Regional Center of Interface, Sidi Mohammed B...",Fès,morocco
152,12239828500,"Ouarsal, Rachid",Laboratoire DIngénierie des Matériaux Organom...,None,morocco
153,8415913200,"Bali, Brahim","Department of Chemistry, Faculty of Sciences,...",Oujda,morocco
